In [5]:
import spacy
from spacy.lang.en.examples import sentences 
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool

In [6]:
data = pd.read_csv('../ukiyo-e_crawler/meta_data/ukiyo-e_artworks_details.csv')
data['Title'] = data['Title'].astype(str).apply(lambda x: x.lower())
data['Description'] = data['Description'].astype(str).apply(lambda x: x.lower())

data.head(1)

,Image URL,Artist,Title,Date,Details,Source,Description,Similar Prints
0,https://data.ukiyo-e.org/bm/scaled/AN00418884_...,Okumura Masanobu,nan,1711,https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,print. sixth of series of twelve (first two mi...,[]


In [11]:
from concurrent.futures import ThreadPoolExecutor

nlp = spacy.load('en_core_web_lg')

def extract_objects(row):
    texts = [row.Title, row.Description]
    objects = []
    exclude_list = ['original', 'copyright', 'series']
    # target_objects = ['courtesan', 'courtesans']
    target_objects = ['']


    for text in texts:
        doc = nlp(text)
        for token in doc:
            if token.pos_ == 'NOUN' and token.lemma_ not in exclude_list:
                objects.append(token.lemma_)

        for ent in doc.ents:
            if ent.label_ in ['PERSON', 'ORG', 'GPE', 'LOC', 'FAC','PRODUCT'] and ent.text not in exclude_list:
                objects.append(ent.text.lower())

        # for target_object in target_objects:
        #     if target_object in text.lower():
        #         objects.append(target_object)

    return list(set(objects))

def process_dataframe(data):
    with ThreadPoolExecutor(max_workers=70) as executor:
        data['Objects'] = list(executor.map(extract_objects, data.itertuples(index=False)))
    return data

# Sample data
# data = pd.DataFrame({
#     'Title': [
#         'Winter Scene',
#     ],
#     'Description': [
#         '''An actor, likely the Osaka actor Yamashita Kinsaku, portrays a young woman leading a dog through a snowy landscape. This is the winter image from a series of the four seasons. It is subtitled Plum Blossoms in the Snow and the flowers suggest an early spring Kabuki theatrical performance. A cartouche on the bottom says that this print is an original by Masanobu, and the gourd-shaped seal functioned much as copyright marks do today.''',
#     ],
# })

a = process_dataframe(data.head(1000))
print(a)


                                             Image URL              Artist   
0    https://data.ukiyo-e.org/bm/scaled/AN00418884_...    Okumura Masanobu  \
1     https://data.ukiyo-e.org/mfa/scaled/sc220382.jpg    Okumura Masanobu   
2       https://data.ukiyo-e.org/loc/scaled/02127v.jpg    Okumura Masanobu   
3    https://data.ukiyo-e.org/bm/scaled/AN00420719_...    Okumura Masanobu   
4     https://data.ukiyo-e.org/mfa/scaled/sc200151.jpg    Okumura Masanobu   
..                                                 ...                 ...   
995  https://data.ukiyo-e.org/japancoll/scaled/p95-...  Nishikawa Sukenobu   
996   https://data.ukiyo-e.org/met/scaled/DP136955.jpg  Nishikawa Sukenobu   
997      https://data.ukiyo-e.org/mak/scaled/10809.jpg  Nishikawa Sukenobu   
998   https://data.ukiyo-e.org/mfa/scaled/sc200472.jpg  Nishikawa Sukenobu   
999  https://data.ukiyo-e.org/artelino/scaled/16827...  Nishikawa Sukenobu   

                                                 Title         

C:\Users\Ben\AppData\Local\Temp\ipykernel_16736\3077854354.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Objects'] = list(executor.map(extract_objects, data.itertuples(index=False)))
